#Import Libraries

In [1]:
from nltk import word_tokenize, ngrams
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
from google.colab import files
import torch
import random
import torch.nn as nn

#Cross-Entropy Loss Function

In [2]:
target = torch.tensor([1]).long()
print(target)
predicted = torch.tensor( [-2.5, -4.212] ).float()
print(predicted.view(1,2))
lossfxn = nn.CrossEntropyLoss()
loss = lossfxn(predicted.view(1,2), target)
print(loss)

tensor([1])
tensor([[-2.5000, -4.2120]])
tensor(1.8779)


# Upload Files for Training

In [ ]:
def findFiles():
    file1 = files.upload()
    file2 = files.upload()
    return file1, file2

files = findFiles()
print(files)


# Define the Feed-Forward Neural Network Model

In [4]:
# Defining the Feedforward Neural Network
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim)

        # Non-linearity
        self.sigmoid = nn.Sigmoid()

        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Linear function
        out = self.fc1(x)
        out = self.sigmoid(out)
        out = self.fc2(out)
        return out

#Preprocess Text Data

In [5]:
# Preprocessing: Convert Unicode to ASCII and build a category dictionary
import unicodedata
import string

all_letters = string.ascii_letters + " \'\"_.,;-'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn' and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))
print(unicodeToAscii('Ço"b"_\'o\' Është'))

Slusarski
Co"b"_'o' Eshte


#Load Data and Build Bigram Tensors

In [6]:
# Build the category_lines dictionary, a list of words per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

AllLines = ""
for file in files:
    for filename, content in file.items():
        category = os.path.splitext(os.path.basename(filename))[0]
        all_categories.append(category)
        lines = readLines(filename)
        category_lines[category] = lines
        AllLines = AllLines + " " + ' '.join(lines)

n_categories = len(all_categories)

#Bigram Functions

In [7]:
# Function to extract bigrams from text content
def findAllBigrams(content):
    bigrams = list(ngrams(content, 2, pad_right=True, right_pad_symbol=' '))
    noDuplicate = list(dict.fromkeys(bigrams))
    return noDuplicate

# Extract bigrams from the full content
all_bigrams = findAllBigrams(AllLines.lower())

# Initialize the feedforward neural network model with bigram size input
FDM = FeedforwardNeuralNetModel(len(all_bigrams), 10, 2)

#Bigram-to-Index and Tensor Conversion

In [8]:
# Function to find the index of a bigram
def bigramToIndex(bigram):
    if bigram in all_bigrams:
        return all_bigrams.index(bigram)
    else:
        return -1

# Convert a bigram to a tensor
def bigramToTensor(bigram):
    tensor = torch.zeros(1, len(all_bigrams))
    n = bigramToIndex(bigram)
    if n > -1:
        tensor[0][n] = 1
    return tensor

# Convert a sentence (line) to a tensor of bigrams
def lineToTensor(line):
    Linebigrams = list(ngrams(line, 2, pad_right=True, right_pad_symbol=' '))
    tensor = torch.zeros(len(all_bigrams))
    for i in range(len(Linebigrams)):
        n = bigramToIndex(Linebigrams[i])
        if n > -1:
            tensor[n] += 1
    return tensor

#Random Training Generator

In [9]:
# Function to randomly select a training example
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

# Function to randomly select a category and a sentence for training
def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)]).long()
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

# Print random examples
for i in range(2):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = English / line = The ancient tapestry hung in the grand hall, depicting battles fought long ago.
category = Albanian / line = Nje studente nga Universiteti i Shkodres fitoi nje cmim te rendesishem per projektin e saj mbi zhvillimin e turizmit te qendrueshem ne Shqiperi.


#Train the Neural Network

# Cross-Entropy Loss Function

In [10]:
# Define hyperparameters
n_epochs = 100
lr = 0.01

# Cross-Entropy Loss Function
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(FDM.parameters(), lr=0.01)

# Training loop
for epoch in range(1, n_epochs + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    optimizer.zero_grad()  # Clears existing gradients
    output = FDM(line_tensor)  # Forward pass
    loss = criterion(output.view(1, 2), category_tensor)  # Calculate loss
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

    # Print loss every 10 epochs
    if epoch % 10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 10/100............. Loss: 0.5551
Epoch: 20/100............. Loss: 0.7083
Epoch: 30/100............. Loss: 0.5773
Epoch: 40/100............. Loss: 0.7346
Epoch: 50/100............. Loss: 0.5115
Epoch: 60/100............. Loss: 0.7018
Epoch: 70/100............. Loss: 0.5053
Epoch: 80/100............. Loss: 0.6896
Epoch: 90/100............. Loss: 0.6587
Epoch: 100/100............. Loss: 0.6438


# BCEWithLogitsLoss Loss Function

In [10]:
# Define hyperparameters
n_epochs = 100
lr = 0.15

# BCEWithLogitsLoss Loss Function
criterion = nn.BCEWithLogitsLoss()  # Replacing CrossEntropyLoss
optimizer = torch.optim.SGD(FDM.parameters(), lr=lr)

# Function to create one-hot encoded target tensor
def create_one_hot_target(category_index, num_classes=2):
    target = torch.zeros(num_classes)
    target[category_index] = 1  # One-hot encode the target
    return target

# Training loop
for epoch in range(1, n_epochs + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()

    # Convert the category_tensor to one-hot encoding
    category_tensor = create_one_hot_target(all_categories.index(category), 2).float()

    optimizer.zero_grad()  # Clears existing gradients
    output = FDM(line_tensor)  # Forward pass

    # Match the target tensor size with the output tensor size
    loss = criterion(output.view(1, -1), category_tensor.view(1, -1))  # Use BCEWithLogitsLoss
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

    if epoch % 10 == 0:
        print(f'Epoch: {epoch}/{n_epochs}, Loss: {loss.item()}')

Epoch: 10/100, Loss: 0.8916890025138855
Epoch: 20/100, Loss: 0.7226808071136475
Epoch: 30/100, Loss: 0.7862013578414917
Epoch: 40/100, Loss: 0.28589195013046265
Epoch: 50/100, Loss: 0.5265478491783142
Epoch: 60/100, Loss: 0.2409517467021942
Epoch: 70/100, Loss: 0.2111690789461136
Epoch: 80/100, Loss: 0.17957395315170288
Epoch: 90/100, Loss: 0.132507786154747
Epoch: 100/100, Loss: 0.12470125406980515


# KLDivLoss Loss Function

In [10]:
# Define hyperparameters
n_epochs = 100
lr = 0.01

# KLDivLoss Loss Function
criterion = nn.KLDivLoss(reduction='batchmean')  # 'batchmean' gives the correct KL divergence per batch
optimizer = torch.optim.SGD(FDM.parameters(), lr=0.01)

# Function to create one-hot encoded probability target tensor
def create_prob_target(category_index, num_classes=2):
    target = torch.zeros(num_classes)
    target[category_index] = 1  # One-hot encode the target
    target = target / target.sum()  # Ensure the target is a probability distribution
    return target

# Training loop for KLDivLoss
for epoch in range(1, n_epochs + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()

    # Convert the category_tensor to one-hot encoding as a probability distribution
    category_tensor = create_prob_target(all_categories.index(category), 2).float()

    optimizer.zero_grad()  # Clears existing gradients
    output = FDM(line_tensor)  # Forward pass

    # Apply log_softmax to the output to convert it into log probabilities
    log_prob_output = torch.log_softmax(output.view(1, -1), dim=1)

    # Match the target tensor size with the output tensor size
    loss = criterion(log_prob_output, category_tensor.view(1, -1))  # Use KLDivLoss
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

    if epoch % 10 == 0:
        print(f'Epoch: {epoch}/{n_epochs}, Loss: {loss.item()}')

Epoch: 10/100, Loss: 0.8777222633361816
Epoch: 20/100, Loss: 0.7585684061050415
Epoch: 30/100, Loss: 0.5282343626022339
Epoch: 40/100, Loss: 0.6515924334526062
Epoch: 50/100, Loss: 0.601322591304779
Epoch: 60/100, Loss: 0.5922287106513977
Epoch: 70/100, Loss: 0.6694270968437195
Epoch: 80/100, Loss: 0.6262274384498596
Epoch: 90/100, Loss: 0.580669105052948
Epoch: 100/100, Loss: 0.4838118553161621


# SoftMarginLoss Loss Function

In [13]:
# Define hyperparameters
n_epochs = 100
lr = 0.001

# SoftMarginLoss Loss Function
criterion = nn.MultiLabelSoftMarginLoss()  # Use MultiLabelSoftMarginLoss for multi-class tasks
optimizer = torch.optim.SGD(FDM.parameters(), lr=0.001)

# Function to convert category index to -1 and 1 for SoftMarginLoss
def create_binary_target(category_index, num_classes=2):
    target = torch.full((num_classes,), -1)  # Initialize with -1 for all classes
    target[category_index] = 1  # Set the correct class to 1
    return target

# Training loop for SoftMarginLoss
for epoch in range(1, n_epochs + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()

    # Convert the category_tensor to -1 and 1 encoding
    category_tensor = create_binary_target(all_categories.index(category), 2).float()

    # # Print target tensor to verify it
    # print(f'Epoch: {epoch}, Target tensor: {category_tensor}')

    optimizer.zero_grad()  # Clears existing gradients
    output = FDM(line_tensor)  # Forward pass

    # Match the target tensor size with the output tensor size
    loss = criterion(output.view(1, -1), category_tensor.view(1, -1))  # Use SoftMarginLoss
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

    if epoch % 10 == 0:
        print(f'Epoch: {epoch}/{n_epochs}, Loss: {loss.item()}')

Epoch: 10/100, Loss: 0.6314465403556824
Epoch: 20/100, Loss: 0.29672446846961975
Epoch: 30/100, Loss: 0.5980336666107178
Epoch: 40/100, Loss: 0.5915906429290771
Epoch: 50/100, Loss: 0.5747272968292236
Epoch: 60/100, Loss: 0.5958119630813599
Epoch: 70/100, Loss: 0.49480384588241577
Epoch: 80/100, Loss: 0.27307549118995667
Epoch: 90/100, Loss: 0.5221474766731262
Epoch: 100/100, Loss: 0.2614022195339203


#Prediction Function

In [11]:
# Function to predict the language of a new sentence
def predict(input_line, n_predictions=2):
    print('\n> %s' % input_line)
    line_tensor = lineToTensor(input_line)
    with torch.no_grad():
        output = FDM(line_tensor)  # Forward pass for prediction
        print("Output is:")
        print(output)
        predictions = []
        result = torch.argmax(output)
        category_index = result.item()
        print("Predicted Language:", all_categories[category_index])

# Test predictions
print("The prediction for 10 English sentences: ")
predict('The sun dipped below the horizon, painting the sky in shades of pink and orange.')
predict('Olivia laughed as the puppy chased its tail in endless circles.')
predict('The library was quiet except for the soft rustling of turning pages.')
predict('A gentle breeze carried the scent of flowers through the open window.')
predict('The old clock tower chimed, signaling the start of a new day.')
predict('Jacob wrote a letter to his grandmother, telling her about his adventures.')
predict('The waves crashed against the shore, leaving seashells scattered along the beach.')
predict('Sarah carefully folded the paper crane and added it to her growing collection.')
predict('The hikers reached the summit, rewarded with a breathtaking view of the valley below.')
predict('A single raindrop hit the window, followed by a sudden downpour.')
print("\n*******************************************************************************")
print("*******************************************************************************")
print("\nThe prediction for 10 Albanian sentences: ")
predict('Dielli u zhyt nën horizont, duke ngjyrosur qiellin me nuanca rozë dhe portokalli.')
predict('Olivia qeshi ndërsa këlyshi ndiqte bishtin e tij në qarqe të pafundme.')
predict('Biblioteka ishte e qetë përveç fërshëllimës së lehtë të faqeve që ktheheshin.')
predict('Një fllad i lehtë solli aromën e luleve përmes dritares së hapur.')
predict('Kulla e vjetër e orës ra, duke shënuar fillimin e një dite të re.')
predict('Jakobi i shkroi një letër gjyshes së tij, duke i treguar për aventurat e tij.')
predict('Valët përplaseshin në breg, duke lënë guaska të shpërndara përgjatë plazhit.')
predict('Sara palosi me kujdes vinçin prej letre dhe e shtoi në koleksionin e saj që rritej.')
predict('Alpinistët arritën në majë, të shpërblyer me një pamje mahnitëse të luginës poshtë.')
predict('Një pikë e vetme shiu goditi dritaren, e ndjekur nga një rrebesh papritur.')

The prediction for 10 English sentences: 

> The sun dipped below the horizon, painting the sky in shades of pink and orange.
Output is:
tensor([0.1827, 0.3853])
Predicted Language: English

> Olivia laughed as the puppy chased its tail in endless circles.
Output is:
tensor([0.2075, 0.3890])
Predicted Language: English

> The library was quiet except for the soft rustling of turning pages.
Output is:
tensor([0.2078, 0.3881])
Predicted Language: English

> A gentle breeze carried the scent of flowers through the open window.
Output is:
tensor([0.2179, 0.3426])
Predicted Language: English

> The old clock tower chimed, signaling the start of a new day.
Output is:
tensor([0.1978, 0.3899])
Predicted Language: English

> Jacob wrote a letter to his grandmother, telling her about his adventures.
Output is:
tensor([0.2689, 0.3645])
Predicted Language: English

> The waves crashed against the shore, leaving seashells scattered along the beach.
Output is:
tensor([0.1803, 0.4453])
Predicted Lang